In [963]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
fig=plt.figure()
import string
import random
import fancyimpute
from datetime import datetime,date,time
from StringIO import StringIO

In [964]:
dateparse = lambda dates: [pd.datetime.strptime(d, '%Y-%m-%d %H:%M:%S') for d in dates]

In [965]:
train=pd.read_csv('bikesharing/train.csv',parse_dates=['datetime'], date_parser=dateparse)
test=pd.read_csv('bikesharing/test.csv',parse_dates=['datetime'], date_parser=dateparse)

In [966]:
#train

In [967]:
train['year'] = train['datetime'].dt.year
train['month'] = train['datetime'].dt.month
train['day'] = train['datetime'].dt.day
train['hours'] = train['datetime'].dt.hour
test['year'] = test['datetime'].dt.year
test['month'] = test['datetime'].dt.month
test['day'] = test['datetime'].dt.day
test['hours'] = test['datetime'].dt.hour

In [968]:
del train['count']
del train['casual']
del train['holiday']
del test['holiday']
del train['month']
del test['month']
del train['day']
del test['day']
del train['temp']
del test['temp']

In [969]:
train.head()

,datetime,season,workingday,weather,atemp,humidity,windspeed,registered,year,hours
0,2011-01-01 00:00:00,1,0,1,14.395,81,0.0,13,2011,0
1,2011-01-01 01:00:00,1,0,1,13.635,80,0.0,32,2011,1
2,2011-01-01 02:00:00,1,0,1,13.635,80,0.0,27,2011,2
3,2011-01-01 03:00:00,1,0,1,14.395,75,0.0,10,2011,3
4,2011-01-01 04:00:00,1,0,1,14.395,75,0.0,1,2011,4


In [970]:
import math
from sklearn.cross_validation import train_test_split

X_all = train.drop(['registered', 'datetime'], axis=1)
y_all = (train['registered'])

num_test = 0.20
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=23)

In [971]:
from sklearn.ensemble import RandomForestRegressor
# Choose the type of classifier. 
clf = RandomForestRegressor(n_estimators=30,max_depth=10)

# Choose some parameter combinations to try
parameters = {'n_estimators': [4, 6, 9], 
              'max_features': ['log2', 'sqrt','auto'], 
              'criterion': ['entropy', 'gini'],
              'max_depth': [2, 3, 5, 10], 
              'min_samples_split': [2, 3, 5],
              'min_samples_leaf': [1,5,8]
             }
# Fit the best algorithm to the data. 
clf.fit(X_train, y_train)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=30, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [972]:
clf.score(X_train,y_train)

0.95075639073409934

In [973]:
clf.score(X_test,y_test)

0.92676069027990837

In [974]:
y_pred = clf.predict(X_test)

In [975]:
def sle(actual, predicted):
    """
    Computes the squared log error.
    This function computes the squared log error between two numbers,
    or for element between a pair of lists or numpy arrays.
    Parameters
    ----------
    actual : int, float, list of numbers, numpy array
             The ground truth value
    predicted : same type as actual
                The predicted value
    Returns
    -------
    score : double or list of doubles
            The squared log error between actual and predicted
    """
    return (np.power(np.log(np.array(y_test)+1) - 
            np.log(np.array(y_pred)+1), 2))

In [976]:
def msle(actual, predicted):
    """
    Computes the mean squared log error.
    This function computes the mean squared log error between two lists
    of numbers.
    Parameters
    ----------
    actual : list of numbers, numpy array
             The ground truth value
    predicted : same type as actual
                The predicted value
    Returns
    -------
    score : double
            The mean squared log error between actual and predicted
    """
    return np.mean(sle(y_test, y_pred))

In [977]:
def rmsle(actual, predicted):
    """
    Computes the root mean squared log error.
    This function computes the root mean squared log error between two lists
    of numbers.
    Parameters
    ----------
    actual : list of numbers, numpy array
             The ground truth value
    predicted : same type as actual
                The predicted value
    Returns
    -------
    score : double
            The root mean squared log error between actual and predicted
    """
    return np.sqrt(msle(y_test, y_pred))


In [978]:
np.sqrt(msle(y_test, y_pred))

0.36825259992687143

In [979]:
X_train.axes

[Int64Index([ 7287,   832,  4646,  3332, 10611,  8958,  1946,  6937,  5134,
              6988,
             ...
              8857,  3674,    39,   347, 10477,  6175,  9704, 10185,  9256,
              8787],
            dtype='int64', length=8708),
 Index([u'season', u'workingday', u'weather', u'atemp', u'humidity',
        u'windspeed', u'year', u'hours'],
       dtype='object')]

In [980]:
clf.feature_importances_

array([ 0.0559746 ,  0.1583773 ,  0.0125044 ,  0.04156292,  0.02340208,
        0.0049522 ,  0.09413105,  0.60909544])

In [981]:
ids = test['datetime']
predictions = clf.predict(test.drop('datetime', axis=1))
output = pd.DataFrame({ 'datetime' : ids, 'registered': predictions })
output.to_csv('registered.csv', index = False)
